<h1> Tabelas Fato</h1>

Vamos começar a trabalhar com as tabelas fato que foram disponibilizadas 

In [140]:
import pandas as pd
import sqlite3 as sql

In [141]:
#importação da tabela fato
arquivo_fact_detalhes = '..\database\FatoDetalhes_DadosModelagem.csv'
arquivo_fact_cabecalho = '..\database\FatoCabecalho_DadosModelagem.txt'
fact_detalhes = pd.read_csv(arquivo_fact_detalhes, sep=';', header=0)
fact_cabecalho = pd.read_table(arquivo_fact_cabecalho, header=0, sep='\t')

In [142]:
#Verificando a tabela detalhes
print(fact_detalhes.head(), fact_detalhes.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2172 entries, 0 to 2171
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CupomID       2172 non-null   int64 
 1   ProdutoID     2172 non-null   int64 
 2   Quantidade    2172 non-null   int64 
 3   Valor         2172 non-null   object
 4   Desconto      2172 non-null   object
 5   Custo         2172 non-null   object
 6   ValorLiquido  2172 non-null   object
dtypes: int64(3), object(4)
memory usage: 118.9+ KB
   CupomID  ProdutoID  Quantidade Valor Desconto   Custo ValorLiquido
0    10308         69           1  2,12        0  1,8317       0,2883
1    10558         73           3  2,97        0  2,4057       0,5643
2    10259         37           1  4,37        0  3,7757       0,5943
3    11077         73           5  6,11   0,0605   5,838       0,2725
4    10752         69           3  6,57        0  5,3217       1,2483 None


Temos 4 Colunas aqui como Object, que deveriam ser Float... Elas não possuem valores nulos de acordo com a info do dataset, como elas estão definidas com "," e não com "." o pandas categoriza elas como string e não como numericos. Vamos substiuir as "," por "." e resolver esse problema 

In [143]:
for value in ['Valor', 'Desconto', 'Custo' , 'ValorLiquido']:
    #alterando "," por "." 
    fact_detalhes[value].replace(',', '.', regex=True, inplace =True)
   
    #Transformando em float e deixando em 2 casas decimais. 
    fact_detalhes[value] = pd.to_numeric(pd.to_numeric(fact_detalhes[value], errors= 'coerce', downcast='float').map(lambda x: '%.2f' % x))

In [159]:
fact_detalhes.info()
#Pegando uma amostra do dataset para visualizarmos 
fact_detalhes.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2172 entries, 0 to 2171
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CupomID          2172 non-null   int64  
 1   ProdutoID        2172 non-null   int64  
 2   Quantidade       2172 non-null   int64  
 3   Valor            2172 non-null   float64
 4   Desconto         2172 non-null   float64
 5   Custo            2172 non-null   float64
 6   ValorLiquido     2172 non-null   float64
 7   Margem_de_lucro  2172 non-null   float64
dtypes: float64(5), int64(3)
memory usage: 135.9 KB


,CupomID,ProdutoID,Quantidade,Valor,Desconto,Custo,ValorLiquido,Margem_de_lucro
2117,10758,52,60,4579.20,0.00,3709.15,870.05,0.1900
443,10266,12,12,128.02,6.10,105.34,22.68,0.1772
53,10278,73,25,26.75,0.00,23.11,3.64,0.1361
94,10288,68,3,38.81,3.53,30.48,8.33,0.2146
1899,10634,18,50,1381.50,0.00,1119.02,262.48,0.1900
524,10909,16,15,148.20,0.00,130.71,17.49,0.1180
886,10842,68,20,265.40,0.00,234.08,31.32,0.1180
1664,10757,64,24,848.40,0.00,732.57,115.83,0.1365
518,10548,41,14,146.72,0.00,118.84,27.88,0.1900
1077,10984,36,40,350.00,0.00,315.85,34.15,0.0976


In [145]:
fact_detalhes.describe()

,CupomID,ProdutoID,Quantidade,Valor,Desconto,Custo,ValorLiquido
count,2172.000000,2172.000000,2172.000000,2172.000000,2172.000000,2172.000000,2172.000000
mean,10662.680479,40.710866,23.967311,820.970622,49.608494,653.911980,167.058439
std,243.294300,22.175948,19.189360,2002.528358,226.864965,1577.331298,455.554335
min,10248.000000,1.000000,1.000000,2.120000,0.000000,1.830000,0.270000
25%,10452.000000,22.000000,10.000000,154.260000,0.000000,123.350000,25.185000
50%,10659.500000,41.000000,20.000000,355.070000,0.000000,282.275000,63.580000
75%,10868.250000,60.000000,30.000000,796.400000,26.835000,648.237500,148.115000
max,11083.000000,77.000000,150.000000,43755.120000,7292.520000,32160.010000,11595.110000


Tudo nos seus devidos tipo, podemos tirar daqui, a margem de lucro de cara venda. Vamos criar uma nova coluna na nossa tabela fato então 

In [146]:
fact_detalhes['Margem_de_lucro'] = round(1 - (fact_detalhes['Custo']/ fact_detalhes['Valor']),4)

In [147]:
fact_detalhes.sample(15)

,CupomID,ProdutoID,Quantidade,Valor,Desconto,Custo,ValorLiquido,Margem_de_lucro
33,10995,60,4,20.84,0.00,18.38,2.46,0.1180
2148,10732,76,20,8250.00,0.00,6682.50,1567.50,0.1900
1114,11035,35,60,366.60,0.00,323.34,43.26,0.1180
417,11076,19,10,122.25,24.45,86.26,35.99,0.2944
844,11036,59,30,247.80,0.00,218.56,29.24,0.1180
849,10725,55,6,249.60,0.00,202.18,47.42,0.1900
1416,10984,16,55,552.75,0.00,487.53,65.22,0.1180
745,11040,21,20,209.20,0.00,184.51,24.69,0.1180
740,10974,63,10,207.40,0.00,182.93,24.47,0.1180
856,10740,28,5,252.66,42.11,170.55,82.11,0.3250


Fato Detalhes Transformada, ajustada e limpinha para ser utilizada, com uma coluna nova de margem para adicionarmos a nossas informações do dataset. 

<h1> fact_cabecalho </h1>

Vamos agora visualizar os dados da cabecalho e fazer seu ETL

In [148]:
print(fact_cabecalho.head(), fact_cabecalho.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 836 entries, 0 to 835
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Data           836 non-null    object
 1   ClienteID      836 non-null    int64 
 2   FuncionarioID  836 non-null    int64 
 3   ValorFrete     836 non-null    object
 4   CupomID        836 non-null    int64 
 5   EmpresaFrete   836 non-null    object
 6   DataEntrega    836 non-null    object
dtypes: int64(3), object(4)
memory usage: 45.8+ KB
         Data  ClienteID  FuncionarioID ValorFrete  CupomID     EmpresaFrete  \
0  22/03/2018         78             11      12,45    11081  Freight Express   
1  21/11/2016         62              3      20,01    10637       One Liners   
2  16/09/2017         64              3      20,36    11019       One Liners   
3  02/07/2017         80             11      20,53    10982  Freight Express   
4  16/09/2016         28              5      20,61    1032

Temos as colunas sem nulos, mas com as tipagens incorretas. Vamos trabalhar as tipagens de data primeiro, e após ajustamos as de numero


In [149]:
for data in ['DataEntrega', 'Data']:
    #alterando o tipo data!
    fact_cabecalho[data] = pd.to_datetime(fact_cabecalho[data]
                                            ,dayfirst=True
                                            )
    

In [150]:
fact_cabecalho.info()
fact_cabecalho.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 836 entries, 0 to 835
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Data           836 non-null    datetime64[ns]
 1   ClienteID      836 non-null    int64         
 2   FuncionarioID  836 non-null    int64         
 3   ValorFrete     836 non-null    object        
 4   CupomID        836 non-null    int64         
 5   EmpresaFrete   836 non-null    object        
 6   DataEntrega    836 non-null    datetime64[ns]
dtypes: datetime64[ns](2), int64(3), object(2)
memory usage: 45.8+ KB


,Data,ClienteID,FuncionarioID,ValorFrete,CupomID,EmpresaFrete,DataEntrega
224,2016-03-18,82,11,"35,9",10401,Freight Express,2016-04-01
531,2015-01-31,51,11,"58,76",10339,Freight Express,2015-02-06
273,2017-04-27,39,9,"39,15",10849,One Liners,2017-05-04
427,2017-06-29,8,14,"50,83",10970,Ship for U,2017-07-05
665,2016-10-26,79,68,"67,94",10608,One Liners,2016-11-14


In [151]:
#alterando "," por "." 
fact_cabecalho['ValorFrete'].replace(',', '.', regex=True, inplace =True)
   
#Transformando em float e deixando em 2 casas decimais. 
fact_cabecalho['ValorFrete'] = pd.to_numeric(pd.to_numeric(fact_cabecalho['ValorFrete'], errors= 'coerce', downcast='float').map(lambda x: '%.2f' % x))

In [152]:
fact_cabecalho.info()
fact_cabecalho.sample(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 836 entries, 0 to 835
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Data           836 non-null    datetime64[ns]
 1   ClienteID      836 non-null    int64         
 2   FuncionarioID  836 non-null    int64         
 3   ValorFrete     836 non-null    float64       
 4   CupomID        836 non-null    int64         
 5   EmpresaFrete   836 non-null    object        
 6   DataEntrega    836 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(3), object(1)
memory usage: 45.8+ KB


,Data,ClienteID,FuncionarioID,ValorFrete,CupomID,EmpresaFrete,DataEntrega
267,2015-09-05,44,68,38.79,10522,One Liners,2015-09-15
804,2018-02-02,25,7,77.29,10342,One Liners,2018-02-14
770,2016-08-07,23,56,75.34,10763,Ship for U,2016-08-08
832,2017-02-10,1,7,79.64,10349,One Liners,2017-02-18
129,2017-06-21,4,9,29.59,10953,One Liners,2017-07-03
368,2015-08-03,28,14,46.26,10551,Ship for U,2015-08-14
810,2017-11-12,1,7,77.65,10277,One Liners,2017-11-23
400,2015-08-13,35,3,48.66,10498,One Liners,2015-08-16
65,2017-03-22,44,68,25.14,10781,One Liners,2017-04-09
619,2017-02-20,23,56,65.51,10604,Ship for U,2017-02-27


In [155]:
#Inserindo dataset no banco de dados
database_name = 'refera'
conn = sql.connect('..\\banco_de_dados\\refera.db')


In [156]:
#INSERINDO NO BANCO DE DADOS A TABELA FACT_CABECALHO
fact_cabecalho.to_sql('FACT_CABECALHO', conn, if_exists='replace', index= False)

836

In [158]:
#INSERINDO NO BANCO DE DADOS A TABELA FACT_DETALHES
fact_detalhes.to_sql('FACT_DETALHES', conn, if_exists='replace', index= False)

2172